<a href="https://colab.research.google.com/github/cccaaannn/deep_learning_colab/blob/master/autoencoder_denoise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Download and unzip dataset

In [ ]:
!wget https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip
!unzip kagglecatsanddogs_3367a.zip

Install extra libs

In [ ]:
!pip install imagepreprocessing

In [1]:
from keras.layers import Dense, Flatten, Reshape, Input, InputLayer
from keras.models import Sequential, Model
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# imagepreprocessing
from  imagepreprocessing.keras_functions import create_training_data_keras, make_prediction_from_array_keras
from  imagepreprocessing.utilities import create_confusion_matrix, train_test_split

set options

In [14]:
img_shape = (32,32,3)
bottleneck_size = 1000
nr_epochs = 20

helper funcitons

In [4]:
def apply_gaussian_noise(data, sigma=0.1):
    noise = np.random.normal(loc=0.0, scale=sigma, size=data.shape)
    return data + noise

def predict_and_show(img, encoder, decoder):
    """Draws original, encoded and decoded images"""
    # img[None] will have shape of (1, 32, 32, 3) which is the same as the model input
    code = encoder.predict(img[None])[0]
    reco = decoder.predict(code[None])[0]

    plt.subplot(1,3,1)
    plt.title("Original")
    plt.imshow(img)

    plt.subplot(1,3,2)
    plt.title("Code")
    plt.imshow(code.reshape([code.shape[-1]//2,-1]))

    plt.subplot(1,3,3)
    plt.title("Reconstructed")
    plt.imshow(reco)
    plt.show()

create training data

In [ ]:
train_x, _, test_x, _  = create_training_data_keras("/content/PetImages", image_size = img_shape[0], percent_to_use=0.1, validation_split=0.2, grayscale=False)

create autoencoder

In [15]:
# The encoder
encoder = Sequential()
encoder.add(InputLayer(img_shape))
encoder.add(Flatten())
encoder.add(Dense(bottleneck_size))

# The decoder
decoder = Sequential()
decoder.add(InputLayer((bottleneck_size,)))
decoder.add(Dense(np.prod(img_shape))) # np.prod(img_shape) is the same as 32*32*3, it's more generic than saying 3072
decoder.add(Reshape(img_shape))


input = Input(img_shape)
bottleneck = encoder(input)
reconstruction = decoder(bottleneck)
autoencoder = Model(input, reconstruction)

autoencoder.compile(optimizer='adamax', loss='mse')

train

In [ ]:
history = autoencoder.fit(x = train_x, y = train_x, epochs = nr_epochs, validation_data=[test_x, test_x])

train for denoising

In [ ]:
for i in range(nr_epochs):
    print("Epoch {0}/{1}, generating noised images".format(i+1, nr_epochs))
    train_x_noise = apply_gaussian_noise(train_x)
    test_x_noise = apply_gaussian_noise(test_x)

    autoencoder.fit(x=train_x_noise, y=train_x, epochs=1, validation_data=[test_x_noise, test_x])

predict

In [ ]:
for i in range(5):
    img = test_x[i]
    predict_and_show(img, encoder, decoder)

predict for noised images

In [ ]:
test_x_noise = apply_gaussian_noise(test_x)
for i in range(5):
    img = test_x_noise[i]
    predict_and_show(img, encoder, decoder)